### Deploy and Interact with Consent based Smart Contract
The aim of this notebook is to compile, deploy and interact with consent based smart contract

### Compile contract from file

In [1]:
# Import libraries
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

In [2]:
# Read in LUCE contract code
with open('./data/ConsentContract.sol', 'r') as file:
    contract_source_code = file.read()

In [3]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [4]:
#compiled_sol

In [5]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:ConsentCode']

In [6]:
#contract_interface

In [7]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [8]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [9]:
# Use local Ganache GUI on macOS
#w3 = Web3(Web3.HTTPProvider("HTTP://192.168.72.1:7545"))

In [10]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [11]:
w3.eth.accounts[0]

'0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA'

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [12]:
# Create contract blueprint
Contract = w3.eth.contract(abi=abi, bytecode=bytecode)

In [13]:
# Submit the transaction that deploys the contract
tx_hash = Contract.constructor().transact()

In [14]:
tx_hash

HexBytes('0x4769d671ec1eae459f286412d7292816c719a8bc04e46acfb8156ca3e6106031')

### Obtain Transcation Receipt

In [15]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [16]:
# We obtain the block number under which it is deployed 
global contract_block
contract_block = w3.eth.blockNumber
print("The contract is deployed with block number",contract_block,".")

The contract is deployed with block number 396 .


In [17]:
# With obtain the final address of the contract 
global contract_address
contract_address = tx_receipt.contractAddress
print("The contract has the address", contract_address,".")

The contract has the address 0xaF5b622e6D0e2645D246e89A9fB2dBa4FaD1F450 .


### Interact with contract

In [18]:
# Create python instance of deployed contract
Contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [19]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts

In [20]:
accounts

['0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA',
 '0xbf7ca7e607516Cd0F1b94a6C77F45fbFD936949C',
 '0x43e196C418b4b7Ebf71ba534042cC8907bd39dc9',
 '0x2aD1e0fAC9d6D523Fb1f21aE1Bc6acA09dD60c7F',
 '0x3c93550555cDD5218bB2923008b3Fb4ea6Ee4618',
 '0xf12C911960968c187090b52998a3597b35BC2619',
 '0x172bE1ee3f659A83a9C307fafafA51c9F91954Ae',
 '0x515f90440fA63377d7451E58B3be334F9393Ad74',
 '0x27F82fD9E21a3758737Cf3BCCA9652064186107C',
 '0x6eEA265cd899aE17Cf588744E4C7Fe31e3258E85',
 '0xE17087878F32b8dAba31B9c7A2c8de6e7DE1478E',
 '0x5F180b2f4750258566513c63381F04105bAAceeD',
 '0x2Ab8E2Df1ce02ECf6d0D0a34ee1e3d7470ebB5ff',
 '0x38643b8d4A129c011F71B934f213582F19aFA04D',
 '0xce4af93d4BA5CED45f3ec13c41000a6a00D67F6F',
 '0xeA9C047468d4ee0c806F364150c7b1d709FfE2A1']

**UploadData**  
```solidity
   function UploadDataPrimaryCategory(
        address _address1,
        bool _NoRestrictions,
        bool _OpenToGeneralResearchAndClinicalCare,
        bool _OpenToHMBResearch,
        bool _OpenToPopulationAndAncestryResearch,
        bool _OpenToDiseaseSpecific)
```
```solidity
    function UploadDataSecondaryCategory(
        address _address1, 
        bool _OpenToGeneticStudiesOnly,
        bool _ResearchSpecificRestrictions,
        bool _OpenToResearchUseOnly,
        bool _NoGeneralMethodResearch)
```

```solidity
   function UploadDataRequirements(
        address _address1,
        bool _GeographicSpecificRestriction,
        bool _OpenToNonProfitUseOnly,
        bool _PublicationRequired,
        bool _CollaborationRequired,
        bool _EthicsApprovalrequired,
        bool _TimeLimitOnUse,
        bool _CostOnUse,
        bool _DataSecurityMeasuresRequired)
```

In [21]:
#Data provider consent (General research & non-profit)
#D-001 (open)
test_address0 = accounts[0]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address0, False, True, False, False, False, False, False, False).transact()

In [22]:
tx_hash

HexBytes('0x8b98b32da4eeedc6fcd41d9c8d1cb538a910bdecf16ba5e8e965289b493f19c4')

**giveResearchPurpose**  
```solidity
 function giveResearchPurpose(
        address _address2,
        bool _UseForMethodsDevelopment,
        bool _UseForReferenceOrControlMaterial,
        bool _UseForPopulationsResearch,
        bool _UseForAncestryResearch,
        bool _UseForHMBResearch)
```

**giveHMBPurpose**  
```solidity
 function giveHMBPurpose( 
        address _address2, 
        bool _UseForFundamentalBioResearch,
        bool _UseForGeneticsResearch,
        bool _UseForDrugDevelopmentResearch, 
        bool _UseForAnyDiseaseResearch,
        bool _UseForAgeCategoriesResearch,
        bool _UseForGenderCategoriesResearch)
```

**giveClinicalPurpose**  
```solidity
 function giveClinicalPurpose( 
        address _address2, 
        bool _UseForDecisionSupport, 
        bool _UseForDiseaseSupport)
```

**givePerson**  
```solidity
 function givePerson( 
        address _address2,
        bool _UseByAcademicProfessionals,
        bool _UseByClinicalProfessionals,
        bool _UseByProfitMakingProfessionals, 
        bool _UseByNonProfessionals)
```

**giveGeographicSpecificRestriction**  
```solidity
 function giveGeographicSpecificRestriction( 
        address _address2, 
        bool _UseBySpecifiedCountries)
```
**giveProfit**  
```solidity
 function giveProfit( 
        address _address2, 
        bool _UseForProfitPurpose,
        bool _UseForNonProfitPurpose)
```
**giveDataRequester_Terms**  
```solidity
 function giveDataRequester_Terms( 
        address _address2, 
        bool _NoTimelineRestrictions,
        bool _NoFormalApprovalRequired,
        bool _NoCollaborationRequired,
        bool _NoPublicationRequired,
        bool _NoDataSecurityMeasures,
        bool _NoDataDestructionRequired,
        bool _NoLinkingOfAccessedRecords,
        bool _NoRecontactingDataSubjects,
        bool _NoIntellectualPropertyClaims,
        bool _NoUseOfAccessedResources,
        bool _NoFeesForAccess)
```

In [23]:
#Data Requesters Account  
test_address13 = accounts[13]
test_address14 = accounts[14]
test_address15 = accounts[15]

#Data Requester 1 (General Research and Profit-Making)

tx_hash = Contract.functions.giveResearchPurpose(test_address13, True, True, False, False, False).transact()
tx_hash = Contract.functions.givePerson(test_address13, True, False, True, False).transact()
tx_hash = Contract.functions.giveProfit(test_address13, True, False).transact()

#Data Requester 2 (HMB and Profit Making)
tx_hash = Contract.functions.giveHMBPurpose(test_address14, True, True, True, False, True, True).transact()
tx_hash = Contract.functions.givePerson(test_address14, False, True, True, False).transact()
tx_hash = Contract.functions.giveProfit(test_address14, True, False).transact()


#Data Requester 3 (Disease Specific and Non-Profit-Making)
tx_hash = Contract.functions.giveHMBPurpose(test_address15, False, False, False, True, False, False).transact()
tx_hash = Contract.functions.givePerson(test_address15, False, True, False, False).transact()
tx_hash = Contract.functions.giveProfit(test_address15, False, True).transact()


In [24]:
Contract.functions.AccessData(test_address0, test_address13).call()

False

In [25]:
Contract.functions.AccessData(test_address0, test_address14).call()

False

In [26]:
Contract.functions.AccessData(test_address0, test_address15).call()

True

In [27]:
#Data provider consent (General research & profit making)
#D-002
test_address1 = accounts[1]
tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address1, False, True, True, True, True).transact()

In [28]:
Contract.functions.AccessData(test_address1, test_address13).call()

True

In [29]:
Contract.functions.AccessData(test_address1, test_address14).call()

True

In [30]:
Contract.functions.AccessData(test_address1, test_address15).call()

False

In [31]:
#Data provider consent (General research & non-profit making)
#H-001 
test_address2 = accounts[2]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address2, False, True, False, False, False).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address2, False, True, False, False, False, False, False, False).transact()

In [32]:
Contract.functions.AccessData(test_address2, test_address13).call()

False

In [33]:
Contract.functions.AccessData(test_address2, test_address14).call()

False

In [34]:
Contract.functions.AccessData(test_address2, test_address15).call()

False

In [35]:
#Data provider consent (General research & profit making)
#H-002 
test_address3 = accounts[3]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address3, False, True, False, False, False).transact()

In [36]:
Contract.functions.AccessData(test_address3, test_address13).call()

True

In [37]:
Contract.functions.AccessData(test_address3, test_address14).call()

False

In [38]:
Contract.functions.AccessData(test_address3, test_address15).call()

False

In [39]:
#Data provider consent (HMB research & non-profit making)
#D-003 
test_address4 = accounts[4]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address4, False, False, True, False, True).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address4, False, True, False, False, False, False, False, False).transact()

In [40]:
Contract.functions.AccessData(test_address4, test_address13).call()

False

In [41]:
Contract.functions.AccessData(test_address4, test_address14).call()

False

In [42]:
Contract.functions.AccessData(test_address4, test_address15).call()

True

In [43]:
#Data provider consent (HMB research & profit making)
#D-004 
test_address5 = accounts[5]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address5, False, False, True, False, True).transact()

In [44]:
Contract.functions.AccessData(test_address5, test_address13).call()

False

In [45]:
Contract.functions.AccessData(test_address5, test_address14).call()

True

In [46]:
Contract.functions.AccessData(test_address5, test_address15).call()

False

In [47]:
#Data provider consent (HMB research & non-profit making)
#H-003 
test_address6 = accounts[6]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address6, False, False, True, False, False).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address6, False, True, False, False, False, False, False, False).transact()

In [48]:
Contract.functions.AccessData(test_address6, test_address13).call()

False

In [49]:
Contract.functions.AccessData(test_address6, test_address14).call()

False

In [50]:
Contract.functions.AccessData(test_address6, test_address15).call()

False

In [51]:
#Data provider consent (HMB research & profit making)
#H-004 
test_address7 = accounts[7]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address7, False, False, True, False, False).transact()

In [52]:
Contract.functions.AccessData(test_address7, test_address13).call()

False

In [53]:
Contract.functions.AccessData(test_address7, test_address14).call()

True

In [54]:
Contract.functions.AccessData(test_address7, test_address15).call()

False

In [55]:
#Data provider consent (Disease-specific research & non-profit making)
#D-005 
test_address8 = accounts[8]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address8, False, False, False, False, True).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address8, False, True, False, False, False, False, False, False).transact()

In [56]:
Contract.functions.AccessData(test_address8, test_address13).call()

False

In [57]:
Contract.functions.AccessData(test_address8, test_address14).call()

False

In [58]:
Contract.functions.AccessData(test_address8, test_address15).call()

True

In [59]:
#Data provider consent (Disease-specific research & profit making)
# D-006 
test_address9 = accounts[9]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address9, False, False, False, False, True).transact()

In [60]:
Contract.functions.AccessData(test_address9, test_address13).call()

False

In [61]:
Contract.functions.AccessData(test_address9, test_address14).call()

False

In [62]:
Contract.functions.AccessData(test_address9, test_address15).call()

False

In [63]:
#Data provider consent (Disease-specific research & non-profit making)
#H-005 
test_address10 = accounts[10]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address10, False, False, False, False, False).transact()
tx_hash = Contract.functions.UploadDataRequirements(test_address10, False, True, False, False, False, False, False, False).transact()

In [64]:
Contract.functions.AccessData(test_address10, test_address13).call()

False

In [65]:
Contract.functions.AccessData(test_address10, test_address14).call()

False

In [66]:
Contract.functions.AccessData(test_address10, test_address15).call()

False

In [67]:
#Data provider consent (Disease-specific research & profit making)
#H-006 
test_address11 = accounts[11]

tx_hash = Contract.functions.UploadDataPrimaryCategory(test_address11, False, False, False, False, False).transact()

In [68]:
Contract.functions.AccessData(test_address11, test_address13).call()

False

In [69]:
Contract.functions.AccessData(test_address11, test_address14).call()

False

In [70]:
Contract.functions.AccessData(test_address11, test_address15).call()

False